In [1]:
import pandas as pd
import pickle

In [2]:
# Load history datas and fixture datas.
df_euro = pd.read_csv("uefa_euro_all_data.csv")
df_fixture = pd.read_csv("uefa_euro_fixture.csv")

# We already saved a pickle file, let's load it as dict_table.
with open('uefa_euro_dict_table', 'rb') as input_file:
    dict_table = pickle.load(input_file)

In [3]:
# After web scrapping, there are some space in datas, cleansing with strip function.
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

df_fixture.head()

,home,score,away,year
0,Germany,Match 1,Scotland,2024
1,Hungary,Match 2,Switzerland,2024
2,Germany,Match 14,Hungary,2024
3,Scotland,Match 13,Switzerland,2024
4,Switzerland,Match 25,Germany,2024


In [4]:
"""Now, we have all historical datas but there are some problem in some of the matches, for example:
'Czechoslovakia 3–1 (a.e.t.) Netherlands' a.e.t means "Extra time" and "After Extra Time". We can remove
this to make all columns the same."""
df_euro['score'] = df_euro['score'].str.replace(' \(a.e.t.\)', '', regex=True)

df_euro.sort_values('year', inplace=True)

In [5]:
df_euro['home'].value_counts()

home
France                  25
Netherlands             23
Italy                   22
Portugal                20
Germany                 18
Spain                   18
England                 17
Denmark                 15
Sweden                  14
Czech Republic          13
Croatia                 12
Belgium                 11
Switzerland             10
Russia                  10
Greece                   9
West Germany             9
Turkey                   8
Romania                  7
Ukraine                  7
Poland                   6
Hungary                  6
Austria                  5
Scotland                 5
Wales                    5
Republic of Ireland      4
Czechoslovakia           4
Soviet Union             2
Bulgaria                 2
Slovenia                 2
FR Yugoslavia            2
Finland                  2
Slovakia                 2
Iceland                  2
Albania                  1
Northern Ireland         1
CIS                      1
Yugoslavia             

In [6]:
"""
In wikipedia, you can see this description.

From 1972 to 1988, Germany competed as West Germany.
From 1960 to 1980, both Slovakia and the Czech Republic competed as Czechoslovakia.[54][55][56][57]
From 1960 to 1984, Serbia competed as Yugoslavia, and in 2000 as FR Yugoslavia.
FR Yugoslavia were initially to appear in 1992 (after qualifying as Yugoslavia), but were replaced after being banned by the United Nations from all international sport.

Due to these clarifications, I want to update the names of some countries.
"""

df_euro['home'] = df_euro['home'].str.strip()
df_euro['away'] = df_euro['away'].str.strip()

df_euro['home'] = df_euro['home'].replace('West Germany', 'Germany')
#df_euro['home'] = df_euro['home'].replace('Northern Ireland', 'Republic of Ireland')
df_euro['home'] = df_euro['home'].replace('Czechoslovakia', 'Czech Republic')

df_euro['away'] = df_euro['away'].replace('West Germany', 'Germany')
#df_euro['away'] = df_euro['away'].replace('Northern Ireland', 'Republic of Ireland')
df_euro['away'] = df_euro['away'].replace('Czechoslovakia', 'Czech Republic')

delete_index = df_euro[df_euro['home'].str.contains('Yugoslavia') | df_euro['away'].str.contains('Yugoslavia') |
       df_euro['home'].str.contains('CIS') | df_euro['away'].str.contains('CIS') |
       df_euro['home'].str.contains('Soviet Union') | df_euro['away'].str.contains('Soviet Union')].index

df_euro.drop(index=delete_index, inplace=True)

## Cleaning DF Datas

In [7]:
# Cleanning score and home/away columns
df_euro['score'] = df_euro['score'].str.replace('[^\d–]', '', regex=True)

# Splitting score columns into home and away goals and dropping score column
df_euro[['HomeGoals', 'AwayGoals']] = df_euro['score'].str.split('–', expand=True)
df_euro.drop('score', axis=1, inplace=True)

# Renaming columns and changing data types
df_euro.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 'year':'Year'}, inplace=True)
df_euro = df_euro.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# Creating new column "totalgoals"
df_euro['TotalGoals'] = df_euro['HomeGoals'] + df_euro['AwayGoals']
df_euro

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
1,Belgium,Germany,1972,1,2,3
2,Hungary,Belgium,1972,1,2,3
4,Czech Republic,Netherlands,1976,3,1,4
7,Czech Republic,Germany,1976,2,2,4
20,Czech Republic,Italy,1980,1,1,2
...,...,...,...,...,...,...
293,Croatia,Czech Republic,2020,1,1,2
294,England,Scotland,2020,0,0,0
295,Croatia,Scotland,2020,3,1,4
297,Poland,Slovakia,2020,1,2,3


# Calculate Team Strength

In [8]:
df_home = df_euro[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_euro[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Albania,0.333333,1.000000
Austria,0.700000,1.200000
Belgium,1.380952,1.333333
Bulgaria,0.666667,2.166667
Croatia,1.363636,1.272727
Czech Republic,1.314286,1.257143
Denmark,1.200000,1.466667
England,1.371429,0.942857
Finland,0.333333,1.000000


# Poisson Distribution


Theory The Poisson distribution is a probability distribution used to model the number of times an event occurs within a fixed interval of time or space. These events are independent of each other and occur with a constant mean rate. Poisson distribution is often used for modeling rare events.

Formula The probability mass function (PMF) of the Poisson distribution is given by:

$$
P(X=k) = \frac {λ^k*e^−λ}{k!}
$$

Where:

P(X=k) is the probability of 𝑋 being equal to 𝑘 events.

𝜆 is the average number of events (Poisson parameter).

𝑘 is the number of occurrences (0, 1, 2, ...).

𝑒 is Euler's number (approximately 2.71828).

Determining Parameters

𝜆 ℎ𝑜𝑚𝑒 : The expected number of goals scored by the home team.

𝜆 𝑎𝑤𝑎𝑦 : The expected number of goals scored by the away team.
These parameters are calculated based on the teams' past performances. For example:

𝜆 ℎ𝑜𝑚𝑒= Home team’s average goals scored × Away team’s average goals conceded

𝜆 𝑎𝑤𝑎𝑦= Away team’s average goals scored × Home team’s average goals conceded

In [9]:
# Function Predict_point
from scipy.stats import poisson

# Default values
default_goals_scored = df_team_strength['GoalsScored'].mean()
default_goals_conceded = df_team_strength['GoalsConceded'].mean()

def predict_points(home, away):
    # If the home team is not in the index, use default values
    if home not in df_team_strength.index:
        home_goals_scored = default_goals_scored
        home_goals_conceded = default_goals_conceded
    else:
        home_goals_scored = df_team_strength.at[home, 'GoalsScored']
        home_goals_conceded = df_team_strength.at[home, 'GoalsConceded']
        
    # If the away team is not in the index, use default values
    if away not in df_team_strength.index:
        away_goals_scored = default_goals_scored
        away_goals_conceded = default_goals_conceded
    else:
        away_goals_scored = df_team_strength.at[away, 'GoalsScored']
        away_goals_conceded = df_team_strength.at[away, 'GoalsConceded']
    
    # Calculate lambda values for Poisson distribution
    lamb_home = home_goals_scored * away_goals_conceded
    lamb_away = away_goals_scored * home_goals_conceded
    prob_home, prob_away, prob_draw = 0, 0, 0
    
    # Loop through possible number of goals for home team (0 to 10)
    for x in range(0, 11):
        # Loop through possible number of goals for away team (0 to 10)
        for y in range(0, 11):
            # Calculate the joint probability of home team scoring x goals and away team scoring y goals
            p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
            if x == y:
                # Add to draw probability if goals are equal
                prob_draw += p
            elif x > y:
                # Add to home win probability if home team scores more goals
                prob_home += p
            else:
                # Add to away win probability if away team scores more goals
                prob_away += p

    # Calculate expected points for home and away teams
    points_home = 3 * prob_home + prob_draw
    points_away = 3 * prob_away + prob_draw
    return (points_home, points_away)

In [10]:
# Serbia and Georgia has never played in UEFA European Football Championship, that is why they have 0 number in team strength.
# In here these countries team strength will fill from predict_points function.
print(default_goals_scored)
print(default_goals_conceded)

0.9537569068110355
1.3119559437364663


# Testing Function

In [11]:
# Example usage
print(predict_points('Turkey', 'Portugal'))
print(predict_points('Germany', 'Scotland'))
print(predict_points('Hungary', 'Switzerland'))
print(predict_points('Turkey', 'Georgia'))

(0.46249005087571726, 2.373772875897278)
(2.4481605156741395, 0.3495832746107654)
(1.3189281306868499, 1.4566001771841368)
(0.9806475983181392, 1.7685119492126786)


In [12]:
df_fixture[:36]

,home,score,away,year
0,Germany,Match 1,Scotland,2024
1,Hungary,Match 2,Switzerland,2024
2,Germany,Match 14,Hungary,2024
3,Scotland,Match 13,Switzerland,2024
4,Switzerland,Match 25,Germany,2024
5,Scotland,Match 26,Hungary,2024
6,Spain,Match 3,Croatia,2024
7,Italy,Match 4,Albania,2024
8,Croatia,Match 15,Albania,2024
9,Spain,Match 16,Italy,2024


In [13]:
df_fixture[36:]

,home,score,away,year
36,Runner-up Group A,Match 38,Runner-up Group B,2024
37,Winner Group A,Match 37,Runner-up Group C,2024
38,Winner Group C,Match 40,3rd Group D/E/F,2024
39,Winner Group B,Match 39,3rd Group A/D/E/F,2024
40,Runner-up Group D,Match 42,Runner-up Group E,2024
41,Winner Group F,Match 41,3rd Group A/B/C,2024
42,Winner Group E,Match 43,3rd Group A/B/C/D,2024
43,Winner Group D,Match 44,Runner-up Group F,2024
44,Winner Match 39,Match 45,Winner Match 37,2024
45,Winner Match 41,Match 46,Winner Match 42,2024


In [14]:
# There are 6 groups and each group has 4 teams so there will be 36 mathches in group stage. 
df_fixture_group_36 = df_fixture[:36].copy()
df_fixture_knockout = df_fixture[36:44].copy()
df_fixture_quarter = df_fixture[44:48].copy()
df_fixture_semi = df_fixture[48:50].copy()
df_fixture_final = df_fixture[50:].copy()

In [15]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Germany (H),0,0,0,0,0,0,0,0
1,2,Scotland,0,0,0,0,0,0,0,0
2,3,Hungary,0,0,0,0,0,0,0,0
3,4,Switzerland,0,0,0,0,0,0,0,0


In [16]:
# In group A, Germany has (H) because they are host of 2024 football championship, so I remove this from dict_table.
dict_table['Group A'] = dict_table['Group A'].replace('Germany (H)', 'Germany')

In [17]:
for group in dict_table:
    # Get the teams in the current group
    teams_in_group = dict_table[group]['Team'].values
    
    # Filter fixtures to only include matches where the home team is in the current group
    df_fixture_group_6 = df_fixture_group_36[df_fixture_group_36['home'].isin(teams_in_group)]
    
    # Iterate over the filtered fixtures
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        
        # Predict points for home and away teams
        points_home, points_away = predict_points(home, away)
        
        # Update the points for the home team
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        
        # Update the points for the away team
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
    
    # Sort teams in the group by points in descending order
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index(drop=True)
    
    # Keep only the 'Team' and 'Pts' columns
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    
    # Round the points to the nearest whole number
    dict_table[group] = dict_table[group].round(0)

In [18]:
dict_table['Group A']

,Team,Pts
0,Germany,7.0
1,Switzerland,4.0
2,Hungary,4.0
3,Scotland,2.0


In [19]:
dict_table['Group B']

,Team,Pts
0,Italy,5.0
1,Spain,5.0
2,Croatia,4.0
3,Albania,2.0


In [20]:
dict_table['Group C']

,Team,Pts
0,England,6.0
1,Denmark,4.0
2,Serbia,4.0
3,Slovenia,3.0


In [21]:
dict_table['Group D']

,Team,Pts
0,France,6.0
1,Netherlands,5.0
2,Poland,3.0
3,Austria,2.0


In [22]:
dict_table['Group E']

,Team,Pts
0,Belgium,7.0
1,Romania,4.0
2,Ukraine,3.0
3,Slovakia,3.0


In [23]:
dict_table['Group F']

,Team,Pts
0,Portugal,6.0
1,Czech Republic,5.0
2,Georgia,4.0
3,Turkey,2.0


# Knock Out

In [24]:
""" After group stage, there will be 8 matches with 16 teams. 
In the 38, 39, 41, and 42 index B, C, E, F group leaders will play with the best four teams from group stages 
which is in third rank this is why, I will implement one algorithm to fill this dataframe."""
df_fixture_knockout

,home,score,away,year
36,Runner-up Group A,Match 38,Runner-up Group B,2024
37,Winner Group A,Match 37,Runner-up Group C,2024
38,Winner Group C,Match 40,3rd Group D/E/F,2024
39,Winner Group B,Match 39,3rd Group A/D/E/F,2024
40,Runner-up Group D,Match 42,Runner-up Group E,2024
41,Winner Group F,Match 41,3rd Group A/B/C,2024
42,Winner Group E,Match 43,3rd Group A/B/C/D,2024
43,Winner Group D,Match 44,Runner-up Group F,2024


In [25]:
third_place_teams = []
third_place_groups = []

# Üçüncü sıradaki takımları ve gruplarını belirleyin
for group in dict_table:
    third_place_team = dict_table[group].loc[2, 'Team']
    third_place_teams.append(third_place_team)
    third_place_groups.append(group)

# Üçüncü sıradaki takımları ve gruplarını bir DataFrame'de toplayın
third_place_df = pd.DataFrame({'Team': third_place_teams, 'Group': third_place_groups})

# Üçüncü sıradaki takımları puanlarına göre sıralayın
third_place_df['Pts'] = third_place_df.apply(lambda row: dict_table[row['Group']].loc[dict_table[row['Group']]['Team'] == row['Team'], 'Pts'].values[0], axis=1)
third_place_teams_sorted = third_place_df.sort_values(by='Pts', ascending=False)['Team'].tolist()

print("Üçüncü sıradaki takımlar (puan sırasına göre):", third_place_teams_sorted)

Üçüncü sıradaki takımlar (puan sırasına göre): ['Hungary', 'Croatia', 'Serbia', 'Georgia', 'Poland', 'Ukraine']


In [26]:
third_place_teams = []
third_place_groups = []

# Identify third-place teams and their groups
for group in dict_table:
    third_place_team = dict_table[group].loc[2, 'Team']
    third_place_teams.append(third_place_team)
    third_place_groups.append(group)

# Collect third-place teams and their groups into a DataFrame
third_place_df = pd.DataFrame({'Team': third_place_teams, 'Group': third_place_groups})

# Add points for third-place teams based on their groups
third_place_df['Pts'] = third_place_df.apply(
    lambda row: dict_table[row['Group']].loc[dict_table[row['Group']]['Team'] == row['Team'], 'Pts'].values[0],axis=1)

# Sort third-place teams by their points in descending order
third_place_teams_sorted = third_place_df.sort_values(by='Pts', ascending=False)['Team'].tolist()

# Print sorted third-place teams
print("Third place teams (sorted by points):", third_place_teams_sorted)

Third place teams (sorted by points): ['Hungary', 'Croatia', 'Serbia', 'Georgia', 'Poland', 'Ukraine']


In [27]:
df_fixture_knockout_copy = df_fixture_knockout.copy()

third_place_mappings = {
    '3rd Group A/D/E/F': third_place_teams_sorted[0],
    '3rd Group D/E/F': third_place_teams_sorted[1],
    '3rd Group A/B/C': third_place_teams_sorted[2],
    '3rd Group A/B/C/D': third_place_teams_sorted[3]
}

# df_fixture_knockout'ta yer alan '3rd Group ...' ifadelerini uygun üçüncü sıradaki takımlarla değiştirin
df_fixture_knockout_copy['away'] = df_fixture_knockout_copy['away'].replace(third_place_mappings)
df_fixture_knockout_copy

,home,score,away,year
36,Runner-up Group A,Match 38,Runner-up Group B,2024
37,Winner Group A,Match 37,Runner-up Group C,2024
38,Winner Group C,Match 40,Croatia,2024
39,Winner Group B,Match 39,Hungary,2024
40,Runner-up Group D,Match 42,Runner-up Group E,2024
41,Winner Group F,Match 41,Serbia,2024
42,Winner Group E,Match 43,Georgia,2024
43,Winner Group D,Match 44,Runner-up Group F,2024


In [28]:
# Loop through each group in the dictionary
for group in dict_table:
    # Get the winner (first place) of the group
    group_winner = dict_table[group].loc[0, 'Team']
    # Get the runner-up (second place) of the group
    runners_up = dict_table[group].loc[1, 'Team']
    # Replace placeholders in the knockout fixture DataFrame with actual team names
    df_fixture_knockout_copy.replace({f'Winner {group}': group_winner, f'Runner-up {group}': runners_up}, inplace=True)

# Add a 'winner' column to the knockout fixture DataFrame, initialized with '?'
df_fixture_knockout_copy['winner'] = '?'

# Print the updated knockout fixture DataFrame
print(df_fixture_knockout_copy)

           home     score            away  year winner
36  Switzerland  Match 38           Spain  2024      ?
37      Germany  Match 37         Denmark  2024      ?
38      England  Match 40         Croatia  2024      ?
39        Italy  Match 39         Hungary  2024      ?
40  Netherlands  Match 42         Romania  2024      ?
41     Portugal  Match 41          Serbia  2024      ?
42      Belgium  Match 43         Georgia  2024      ?
43       France  Match 44  Czech Republic  2024      ?


In [29]:
def get_winner(df_fixture_updated):
    # Iterate over each row in the fixture DataFrame
    for index, row in df_fixture_updated.iterrows():
        # Get the home and away team names
        home, away = row['home'], row['away']
        
        # Predict the points for the home and away teams
        points_home, points_away = predict_points(home, away)
        
        # Determine the winner based on predicted points
        if points_home > points_away:
            winner = home
        else:
            winner = away
        
        # Update the 'winner' column in the DataFrame with the determined winner
        df_fixture_updated.loc[index, 'winner'] = winner
    
    # Return the updated DataFrame
    return df_fixture_updated

In [30]:
get_winner(df_fixture_knockout_copy)

,home,score,away,year,winner
36,Switzerland,Match 38,Spain,2024,Spain
37,Germany,Match 37,Denmark,2024,Germany
38,England,Match 40,Croatia,2024,England
39,Italy,Match 39,Hungary,2024,Italy
40,Netherlands,Match 42,Romania,2024,Netherlands
41,Portugal,Match 41,Serbia,2024,Portugal
42,Belgium,Match 43,Georgia,2024,Belgium
43,France,Match 44,Czech Republic,2024,France


# Quarter Final

In [31]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winner {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [32]:
update_table(df_fixture_knockout_copy, df_fixture_quarter)

,home,score,away,year,winner
44,Italy,Match 45,Germany,2024,?
45,Portugal,Match 46,Netherlands,2024,?
46,England,Match 48,Spain,2024,?
47,Belgium,Match 47,France,2024,?


In [33]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
44,Italy,Match 45,Germany,2024,Italy
45,Portugal,Match 46,Netherlands,2024,Portugal
46,England,Match 48,Spain,2024,Spain
47,Belgium,Match 47,France,2024,France


# Semi Final

In [34]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
48,Italy,Match 49,Portugal,2024,?
49,France,Match 50,Spain,2024,?


In [35]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
48,Italy,Match 49,Portugal,2024,Italy
49,France,Match 50,Spain,2024,Spain


# Final

In [36]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
50,Italy,Match 51,Spain,2024,?


In [37]:
get_winner(df_fixture_final)

,home,score,away,year,winner
50,Italy,Match 51,Spain,2024,Italy
